<a href="https://colab.research.google.com/github/botingchen/ML_Project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import requests # to make TMDB API calls
import locale # to format currency as USD
locale.setlocale( locale.LC_ALL, '' )

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter # to format currency on charts axis
import json
import csv
api_key = "c9d26a6782e25df032d62899e2bbe7ef"
# https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key=<<api_key>>&language=en-US
# https://api.themoviedb.org/3/discover/movie?api_key=###&primary_release_date.gte=2006-01-01&primary_release_date.lte=2006-12-31
dataset = []
for year in range(2016,2022):
  for page_num in range(1,51):
    response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=c9d26a6782e25df032d62899e2bbe7ef&year='+ str(year) + '&page=' + str(page_num))
    dataset.append(response.json())
# response = requests.get('https://api.themoviedb.org/3/movie/335797/credits?api_key=c9d26a6782e25df032d62899e2bbe7ef&language=en-US')

In [92]:
# highest_revenue = []
# highest_revenue.append(response_2016_2.json())
# # print(highest_revenue['page'])
#   # print(i)
# for column in highest_revenue:
#   for i in column['results']:
#     print(i)
# # print(highest_revenue)
id = []
original_title = []
genre_ids = []
original_language =[]
popularity = []
release_date = []
for i in range(0,300):
  for data in dataset[i]['results']:
    id.append(data['id'])
    original_title.append(data['original_title'])
    genre_ids.append(data['genre_ids'])
    original_language.append(data['original_language'])
    popularity.append(data['popularity'])
    release_date.append(data['release_date'])
# print(dataset[299])

In [103]:
df = pd.DataFrame(list(zip(id, original_title, genre_ids, original_language, popularity, release_date)),columns =['TMDB_id', 'original_title', 'genre_ids', 'original_language', 'popularity', 'release_date'])
temp_1 = []
### get cast ###
for index in df['TMDB_id']:
  response = requests.get('https://api.themoviedb.org/3/movie/' + str(index) + '/credits?api_key=c9d26a6782e25df032d62899e2bbe7ef&language=en-US')
  temp_1.append(response.json())

In [141]:
cast_name = [None] * 6000
cast_id = [None] * 6000
cast_popularity = [None] * 6000
for iter in range(0,6000):
  count = 0
  cn = []
  ci = []
  cp = []
  for cast in temp_1[iter]['cast']:
    count += 1
    if count == 6:
      continue
    cn.append(cast['original_name'])
    ci.append(cast['id'])
    cp.append(cast['popularity'])
  cast_name[iter] = cn
  cast_id[iter] = ci
  cast_popularity[iter] = cp    
df['cast_name'] = cast_name
df['cast_id'] = cast_id
df['cast_popularity'] = cast_popularity


,TMDB_id,original_title,genre_ids,original_language,popularity,release_date,cast_name,cast_id,cast_popularity
0,102382,The Amazing Spider-Man 2,"[28, 12, 14]",en,1069.261,2014-04-16,"[Andrew Garfield, Emma Stone, Jamie Foxx, Dane...","[37625, 54693, 134, 122889, 10132, 13242, 35, ...","[59.496, 20.223, 11.873, 11.457, 2.659, 10.92,..."
1,557,Spider-Man,"[14, 28]",en,881.486,2002-05-01,"[Tobey Maguire, Willem Dafoe, Kirsten Dunst, J...","[2219, 5293, 205, 17051, 19153, 18999, 20580, ...","[38.197, 27.696, 22.164, 12.303, 3.864, 19.57,..."
2,559,Spider-Man 3,"[14, 28, 12]",en,884.269,2007-05-01,"[Tobey Maguire, Kirsten Dunst, James Franco, T...","[2219, 205, 17051, 19159, 17052, 18998, 18999,...","[38.197, 22.164, 12.303, 11.794, 4.404, 8.422,..."
3,335797,Sing,"[16, 35, 10751, 10402]",en,625.719,2016-11-23,"[Matthew McConaughey, Reese Witherspoon, Seth ...","[10297, 368, 52139, 1245, 4764, 1540615, 12094...","[15.356, 14.74, 6.437, 42.053, 6.211, 6.032, 5..."
4,673,Harry Potter and the Prisoner of Azkaban,"[12, 14]",en,300.910,2004-05-31,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ...","[10980, 10989, 10990, 1923, 5658, 64, 4566, 10...","[30.499, 19.278, 46.894, 11.044, 13.221, 19.54..."


In [142]:
### get revenue budget ...###
temp_2 = []
for index in df['TMDB_id']:
  response = requests.get('https://api.themoviedb.org/3/movie/' + str(index) + '?api_key=c9d26a6782e25df032d62899e2bbe7ef&language=en-US')
  temp_2.append(response.json())

In [155]:
### put budget, runtime into dataframe ###
revenue = []
budget = []
runtime = []

for iter in range(len(temp_2)):
  revenue.append(temp_2[iter]['revenue'])
  budget.append(temp_2[iter]['budget'])
  runtime.append(temp_2[iter]['runtime'])

df['budget'] = budget
df['runtime'] = runtime
df.head()

,TMDB_id,original_title,genre_ids,original_language,popularity,release_date,cast_name,cast_id,cast_popularity,budget,runtime
0,102382,The Amazing Spider-Man 2,"[28, 12, 14]",en,1069.261,2014-04-16,"[Andrew Garfield, Emma Stone, Jamie Foxx, Dane...","[37625, 54693, 134, 122889, 10132, 13242, 35, ...","[59.496, 20.223, 11.873, 11.457, 2.659, 10.92,...",200000000,142.0
1,557,Spider-Man,"[14, 28]",en,881.486,2002-05-01,"[Tobey Maguire, Willem Dafoe, Kirsten Dunst, J...","[2219, 5293, 205, 17051, 19153, 18999, 20580, ...","[38.197, 27.696, 22.164, 12.303, 3.864, 19.57,...",139000000,121.0
2,559,Spider-Man 3,"[14, 28, 12]",en,884.269,2007-05-01,"[Tobey Maguire, Kirsten Dunst, James Franco, T...","[2219, 205, 17051, 19159, 17052, 18998, 18999,...","[38.197, 22.164, 12.303, 11.794, 4.404, 8.422,...",258000000,139.0
3,335797,Sing,"[16, 35, 10751, 10402]",en,625.719,2016-11-23,"[Matthew McConaughey, Reese Witherspoon, Seth ...","[10297, 368, 52139, 1245, 4764, 1540615, 12094...","[15.356, 14.74, 6.437, 42.053, 6.211, 6.032, 5...",75000000,108.0
4,673,Harry Potter and the Prisoner of Azkaban,"[12, 14]",en,300.910,2004-05-31,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ...","[10980, 10989, 10990, 1923, 5658, 64, 4566, 10...","[30.499, 19.278, 46.894, 11.044, 13.221, 19.54...",130000000,141.0


In [232]:
company_name = []
company_id = []
for iter in range(len(temp_2)):
  count = 0
  con = []
  coi = []
  for company in temp_2[iter]['production_companies']:
    count += 1
    if count == 3:
      continue
    con.append(company['name'])
    coi.append(company['id'])
  company_name.append(con)
  company_id.append(coi)
df['company_name'] = company_name
df['company_id'] = company_id
df['revenue'] = revenue

genre_name = []
for iter in range(6000):
  gn = []
  for genre in temp_2[iter]['genres']:
    gn.append(genre['name'])
  genre_name.append(gn)
df['genre_name'] = genre_name

In [236]:
### do eight-level classification on revenue ###
from prettytable import PrettyTable
revenue_sl = [None] * 6000
count_F = 0
count_E = 0
count_D = 0
count_C = 0
count_B = 0
count_A = 0
count_S = 0
count_SS = 0
for i,rev in enumerate(revenue):
  if revenue[i] == 0:
    revenue_sl[i] = 'F'
    count_F += 1
  elif revenue[i] > 0 and revenue[i] <= 10000000:
    revenue_sl[i] = 'E'
    count_E += 1
  elif revenue[i] > 10000000 and revenue[i] <= 50000000:
    revenue_sl[i] = 'D'
    count_D += 1
  elif revenue[i] > 50000000 and revenue[i] <= 100000000:
    revenue_sl[i] = 'C'
    count_C += 1
  elif revenue[i] > 100000000 and revenue[i] <= 500000000:
    revenue_sl[i] = 'B'
    count_B += 1
  elif revenue[i] > 500000000 and revenue[i] <= 1000000000:
    revenue_sl[i] = 'A'
    count_A += 1
  elif revenue[i] > 1000000000 and revenue[i] <= 5000000000:
    revenue_sl[i] = 'S'
    count_S += 1
  elif revenue[i] > 5000000000 and revenue[i] <= 10000000000:
    revenue_sl[i] = 'SS'
    count_SS += 1

table = PrettyTable()
table.field_names = ["Revenue_Level","Count"]
table.add_row(["F",count_F])
table.add_row(["E",count_E])
table.add_row(["D",count_D])
table.add_row(["C",count_C])
table.add_row(["B",count_B])
table.add_row(["A",count_A])
table.add_row(["S",count_S])
table.add_row(["SS",count_SS])
print(table)
df['Revenue_Level'] = revenue_sl
df.head()

+---------------+-------+
| Revenue_Level | Count |
+---------------+-------+
|       F       |  2412 |
|       E       |  842  |
|       D       |  839  |
|       C       |  450  |
|       B       |  1116 |
|       A       |  263  |
|       S       |   78  |
|       SS      |   0   |
+---------------+-------+


,TMDB_id,original_title,genre_ids,original_language,popularity,release_date,cast_name,cast_id,cast_popularity,budget,runtime,company_name,company_id,revenue,Revenue_Level,original_language_label_encode,genre_name
0,102382,The Amazing Spider-Man 2,"[28, 12, 14]",en,1069.261,2014-04-16,"[Andrew Garfield, Emma Stone, Jamie Foxx, Dane...","[37625, 54693, 134, 122889, 10132, 13242, 35, ...","[59.496, 20.223, 11.873, 11.457, 2.659, 10.92,...",200000000,142.0,"[Marvel Entertainment, Columbia Pictures, Avi ...","[7505, 5, 166230]",708962323,A,6,"[Action, Adventure, Fantasy]"
1,557,Spider-Man,"[14, 28]",en,881.486,2002-05-01,"[Tobey Maguire, Willem Dafoe, Kirsten Dunst, J...","[2219, 5293, 205, 17051, 19153, 18999, 20580, ...","[38.197, 27.696, 22.164, 12.303, 3.864, 19.57,...",139000000,121.0,"[Marvel Enterprises, Laura Ziskin Productions]","[19551, 326]",821708551,A,6,"[Fantasy, Action]"
2,559,Spider-Man 3,"[14, 28, 12]",en,884.269,2007-05-01,"[Tobey Maguire, Kirsten Dunst, James Franco, T...","[2219, 205, 17051, 19159, 17052, 18998, 18999,...","[38.197, 22.164, 12.303, 11.794, 4.404, 8.422,...",258000000,139.0,"[Laura Ziskin Productions, Marvel Studios]","[326, 420]",890871626,A,6,"[Fantasy, Action, Adventure]"
3,335797,Sing,"[16, 35, 10751, 10402]",en,625.719,2016-11-23,"[Matthew McConaughey, Reese Witherspoon, Seth ...","[10297, 368, 52139, 1245, 4764, 1540615, 12094...","[15.356, 14.74, 6.437, 42.053, 6.211, 6.032, 5...",75000000,108.0,"[Illumination Entertainment, Universal Picture...","[6704, 33, 1778]",634151679,A,6,"[Animation, Comedy, Family, Music]"
4,673,Harry Potter and the Prisoner of Azkaban,"[12, 14]",en,300.910,2004-05-31,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ...","[10980, 10989, 10990, 1923, 5658, 64, 4566, 10...","[30.499, 19.278, 46.894, 11.044, 13.221, 19.54...",130000000,141.0,"[Warner Bros. Pictures, 1492 Pictures, P of A ...","[174, 436, 19092]",789804554,A,6,"[Adventure, Fantasy]"


In [244]:
rd = df['release_date']
season = []
for month in rd.str.split('-'):
  if month[1] == '01' or month[1] == '02' or month[1] == '12':
    season.append("winter")
  elif month[1] == '03' or month[1] == '04' or month[1] == '05':
    season.append("spring")
  elif month[1] == '06' or month[1] == '07' or month[1] == '08':
    season.append('summer')
  elif month[1] == '09' or month[1] == '10' or month[1] == '11':
    season.append('fall')
df['season'] = season
df.head()



,TMDB_id,original_title,genre_ids,original_language,popularity,release_date,cast_name,cast_id,cast_popularity,budget,runtime,company_name,company_id,revenue,Revenue_Level,original_language_label_encode,genre_name,season
0,102382,The Amazing Spider-Man 2,"[28, 12, 14]",en,1069.261,2014-04-16,"[Andrew Garfield, Emma Stone, Jamie Foxx, Dane...","[37625, 54693, 134, 122889, 10132, 13242, 35, ...","[59.496, 20.223, 11.873, 11.457, 2.659, 10.92,...",200000000,142.0,"[Marvel Entertainment, Columbia Pictures, Avi ...","[7505, 5, 166230]",708962323,A,6,"[Action, Adventure, Fantasy]",spring
1,557,Spider-Man,"[14, 28]",en,881.486,2002-05-01,"[Tobey Maguire, Willem Dafoe, Kirsten Dunst, J...","[2219, 5293, 205, 17051, 19153, 18999, 20580, ...","[38.197, 27.696, 22.164, 12.303, 3.864, 19.57,...",139000000,121.0,"[Marvel Enterprises, Laura Ziskin Productions]","[19551, 326]",821708551,A,6,"[Fantasy, Action]",spring
2,559,Spider-Man 3,"[14, 28, 12]",en,884.269,2007-05-01,"[Tobey Maguire, Kirsten Dunst, James Franco, T...","[2219, 205, 17051, 19159, 17052, 18998, 18999,...","[38.197, 22.164, 12.303, 11.794, 4.404, 8.422,...",258000000,139.0,"[Laura Ziskin Productions, Marvel Studios]","[326, 420]",890871626,A,6,"[Fantasy, Action, Adventure]",spring
3,335797,Sing,"[16, 35, 10751, 10402]",en,625.719,2016-11-23,"[Matthew McConaughey, Reese Witherspoon, Seth ...","[10297, 368, 52139, 1245, 4764, 1540615, 12094...","[15.356, 14.74, 6.437, 42.053, 6.211, 6.032, 5...",75000000,108.0,"[Illumination Entertainment, Universal Picture...","[6704, 33, 1778]",634151679,A,6,"[Animation, Comedy, Family, Music]",fall
4,673,Harry Potter and the Prisoner of Azkaban,"[12, 14]",en,300.910,2004-05-31,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ...","[10980, 10989, 10990, 1923, 5658, 64, 4566, 10...","[30.499, 19.278, 46.894, 11.044, 13.221, 19.54...",130000000,141.0,"[Warner Bros. Pictures, 1492 Pictures, P of A ...","[174, 436, 19092]",789804554,A,6,"[Adventure, Fantasy]",spring


In [256]:
### do label encode on orginal language and one hot on genre_ids ###
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['original_language_label_encode'] = labelencoder.fit_transform(df['original_language'])
df.isnull().sum()
df.fillna(int(df['runtime'].mean()), inplace=True)
df.isnull().sum()
# p_write = df.to_csv('dataset.csv',mode = 'w',index = False)

TMDB_id                           0
original_title                    0
genre_ids                         0
original_language                 0
popularity                        0
release_date                      0
cast_name                         0
cast_id                           0
cast_popularity                   0
budget                            0
runtime                           0
company_name                      0
company_id                        0
revenue                           0
Revenue_Level                     0
original_language_label_encode    0
genre_name                        0
season                            0
dtype: int64

In [258]:
df['genre_name'].unique

<bound method Series.unique of 0                [Action, Adventure, Fantasy]
1                           [Fantasy, Action]
2                [Fantasy, Action, Adventure]
3          [Animation, Comedy, Family, Music]
4                        [Adventure, Fantasy]
                        ...                  
5995                   [Action, History, War]
5996                         [Drama, History]
5997                         [Drama, Romance]
5998                [Action, Science Fiction]
5999    [Animation, Family, Fantasy, Romance]
Name: genre_name, Length: 6000, dtype: object>